Notebook to process the levodopa equivalent dose information from PPMI.

In [ ]:
import numpy as np
import pandas as pd

from calc_ledd import calc_ledd

### Demographics
Use preprocessed demographics. Enrolled patients have an enroll date that is not null.

In [ ]:
path = "data/"### PUT PATH TO DATA HERE ###

In [ ]:
demo = pd.read_csv('demographics.csv',index_col=0) #note that this is the processed data created by 'Demographics_Data_Processing.ipynb
enrol_pat = demo[~demo.ENROLLDT.isnull()] # ENROLLDT  enroll date

In [ ]:
enrol_pat.head(25)

### LEDD Medications

In [ ]:
ledd_med = pd.read_csv(path + 'LEDD_Concomitant_Medication_Log.csv')

In [ ]:
ledd_med.head()

In [ ]:
ledd_med

Data we're intested in PATNO, EVENT_ID, LEDD, STARTDT, STARTEST, STOPDT, STOPEST, CMTRT, CMDOSE, CMDOSU, ONGOING for only PD medications and patients who are enrolled.

In [ ]:
ledd_df = ledd_med.merge(enrol_pat[['APPRDX','PATNO']], on=["PATNO"]) 

In [ ]:
ledd_df

In [ ]:
pt_records = ledd_df.groupby('PATNO')

In [ ]:
counts_per_patient = pt_records.STARTDT.count().value_counts(sort=False)

In [ ]:
counts_per_patient

Calculate LEDD for each patno date combination in the signature form

In [ ]:
signature = pd.read_csv(path + '/Signature_Form-Archived.csv') # new downloaded

In [ ]:
signature.head()

In [ ]:
ledd_processed = pd.DataFrame(columns=['PATNO','EVENT_ID','INFODT','LEDD'])

In [ ]:
ledd_df.STARTDT = pd.to_datetime(ledd_df.STARTDT)
ledd_df.STOPDT = pd.to_datetime(ledd_df.STOPDT)
for idx in signature.index:
    pn = signature.loc[idx].PATNO
    ei = signature.loc[idx].EVENT_ID
    idt = signature.loc[idx].INFODT
    ledd = calc_ledd(ledd_df, pn, pd.to_datetime(idt))
    ledd_processed = ledd_processed.append({'PATNO':pn, 'EVENT_ID':ei, 'INFODT': idt, 'LEDD':ledd},
                                   ignore_index=True)

In [ ]:
ledd_processed.to_csv('ledd.csv')